# 사용자가 업로드한 사진에 대해 Instance Segmentaiton 진행

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
cd drive/MyDrive/2021_capstone/Local_Style_Transfer

[Errno 2] No such file or directory: 'drive/MyDrive/2021_capstone/Local_Style_Transfer'
/content/drive/MyDrive/2021_capstone/Local_Style_Transfer


In [23]:
ls

backbone.py                     ilsvrc2012_wordnet_lemmas.txt  requirements.txt
CHANGELOG.md                    image_similarity.py            results/
chromedriver                    item_recommend.py              run_coco_eval.py
chromedriver.exe                layers/                        scripts/
crawling_image/                 LICENSE                        search_image/
data/                           local_stylized.jpg             topk_sim_image/
entire_local_style_transfer.py  meta-data-files/               train.py
environment.yml                 out.csv                        url.txt
eval.py                         product_recommend.py           utils/
external/                       __pycache__/                   web/
fg_bg/                          pyimagesearch/                 weights/
func_segmentation.py            README.md                      yolact.py


## Pretrained 된 pth file로 segmentation 진행

In [30]:
!python func_segmentation.py --trained_model=weights/yolact/yolact_base_54_800000.pth --score_threshold=0.15 --top_k=15 --image=data/train/images/test_bedroom1.jpg:data/eval_result/test_bedroom1.jpg --display_bbox=True --display_text=True  --display_masks=True

Config not specified. Parsed yolact_base_config from the file name.

/usr/local/lib/python3.7/dist-packages/torch/jit/_recursive.py:235: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))
/usr/local/lib/python3.7/dist-packages/torch/jit/_recursive.py:235: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))
/usr/local/lib/python3.7/dist-packages/torch/jit/_recursive.py:235: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))
Loading model... Done.
bbox_label_list :  [[array([340, 195, 845, 460]), 'bed00'], [array([ 31, 198, 220, 438]), 'chair01'], 

# 사용자가 원하는 object 및 스타일을 선택한 이후 Local Style Transfer

In [ ]:
!python entire_local_style_transfer.py --image_path "./fg_bg/" --targets "bed00, chair01" --vgg_path "./weights/style_transfer/vgg_normalised.pth" --decoder_path "./weights/style_transfer/decoder.pth" --content_image "./data/train/images/test_bedroom1.jpg" --style_image "./data/style/modern/blue.jpg" --output_style_path "./fg_bg/stylized.jpg" --style_intensity "Middle" --fg_image_path "./fg_bg/fg_result.jpg" --bg_image_path "./fg_bg/bg_result.jpg" --stylized_image_path "./fg_bg/stylized.jpg" --output_local_style_path "./local_stylized.jpg"

## 유사 이미지 검색

In [ ]:
!python item_recommend.py --targets "bed00, chair01" --search_img_path "./fg_bg/stylized.jpg" --crawling_img_path "./crawling_image/" --color "blue"

[['./crawling_image/bed00/bed00.jpg', 'https://amazon.com/Utopia-Bedding-Parure-Lit-Microfibre/dp/B07S8RXW4V/ref=sr_1_1?keywords=blue+bed+set&qid=1638894741&sr=8-1', '28,127원'], ['./crawling_image/bed00/bed02.jpg', 'https://amazon.com/Utopia-Bedding-Housse-Couette-imprim%C3%A9e/dp/B078RKXNNN/ref=sr_1_3?keywords=blue+bed+set&qid=1638894741&sr=8-3', '24,107원'], ['./crawling_image/bed00/bed03.jpg', 'https://amazon.com/Utopia-Bedding-Ensemble-Draps-Microfibre/dp/B07SDF5ZRG/ref=sr_1_4?keywords=blue+bed+set&qid=1638894741&sr=8-4', '26,787원']]
[['./crawling_image/chair01/chair00.jpg', 'https://amazon.com/VanderHOME-Slipcover-Polyester-Furniture-Protector/dp/B08MBX91GP/ref=sr_1_1?keywords=blue+sofa+chair&qid=1638894743&sr=8-1', '54,927원'], ['./crawling_image/chair01/chair03.jpg', 'https://amazon.com/HSBAIS-Housse-canap%C3%A9-Cover-Couverture/dp/B07KX1W9SM/ref=sr_1_4?keywords=blue+sofa+chair&qid=1638894743&sr=8-4', '83,067원'], ['./crawling_image/chair01/chair04.jpg', 'https://amazon.com/Nordic-